In [ ]:
# 최석재 lingua@naver.com
# 구글 드라이브와 연결
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# 경로 설정
path = '/content/gdrive/MyDrive/pytest/'
!ls '/content/gdrive/MyDrive/pytest/'
DATA_OUT_PATH = path+'data/'
model_name = 'transformer'

'충북 관광 데이터'     iris.csv			   뉴스데이터_test.csv
'불법 대부업 데이터'   negative.txt		   text_binary_model.h5
 감성대화말뭉치        newfile2.txt		   text_binary_tokenizer.pickle
 alice.png	       newfile.csv		   뉴스데이터_train.csv
 base		       pima-indians-diabetes.csv   김소월시.txt
 BostonHousing.csv     positive.txt		   윤동주시.txt
 chatdata.csv	       ratings_morphed.txt	   금융규제운영규정.txt
 chatdata_small.csv    ratings_small.txt	   vocabulary.txt
 data		       ratings.txt		   wiki_test.txt
 income.csv	       sonar.csv		   wine.csv
 income_test.csv       temp


In [ ]:
# 데이터 확인
# 빠른 진행을 위해 small 데이터로 수행한다.
# header는 제외하고 로딩
import pandas as pd
data = pd.read_csv(path+'chatdata_small.csv', names=['Q', 'A', "label"], sep=',', header=0, encoding='cp949')
print('data length: ', len(data))
print('data sample: ', data.head())

data length:  19
data sample:                   Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


In [ ]:
inputs, outputs = list(data['Q']), list(data['A'])
print("inputs: ", inputs)
print("outputs: ", outputs)

inputs:  ['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네', 'SD카드 망가졌어', 'SD카드 안돼', 'SNS 맞팔 왜 안하지ㅠㅠ', 'SNS 시간낭비인 거 아는데 매일 하는 중', 'SNS 시간낭비인데 자꾸 보게됨', 'SNS보면 나만 빼고 다 행복해보여', '가끔 궁금해', '가끔 뭐하는지 궁금해', '가끔은 혼자인게 좋다', '가난한 자의 설움', '가만 있어도 땀난다', '가상화폐 쫄딱 망함', '가스불 켜고 나갔어', '가스불 켜놓고 나온거 같아']
outputs:  ['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.', '여행은 언제나 좋죠.', '눈살이 찌푸려지죠.', '다시 새로 사는 게 마음 편해요.', '다시 새로 사는 게 마음 편해요.', '잘 모르고 있을 수도 있어요.', '시간을 정하고 해보세요.', '시간을 정하고 해보세요.', '자랑하는 자리니까요.', '그 사람도 그럴 거예요.', '그 사람도 그럴 거예요.', '혼자를 즐기세요.', '돈은 다시 들어올 거예요.', '땀을 식혀주세요.', '어서 잊고 새출발 하세요.', '빨리 집에 돌아가서 끄고 나오세요.', '빨리 집에 돌아가서 끄고 나오세요.']


In [ ]:
# 시작부호와 종료부호 부착
# 데이터가 모두 3종이 필요하다. source 언어에는 encoder_input 1개, target 언어에는 decoder_input, decoder_target 2개
# encoder는 source 언어를 그대로 사용하면 되나, decoder는 seq2seq의 사용을 위해 <sos>, <eos>를 부착해야 한다
# decoder_input 데이터의 시작에는 <sos>, 문장의 끝에는 <eos>를 부착한다
# decoder_target 데이터는 <eos>만 필요하다
# 어절분리가 되도록 <sos> 뒤에 공백, <eos> 앞에 공백을 두어야 한다
outputs_input = data.A.apply(lambda x : '<SOS> '+x+' <EOS>')
outputs_target = data.A.apply(lambda x : x+' <EOS>')
print('\noutputs_input:\n', outputs_input.sample(5))
print("\noutputs_target\n: ", outputs_target.sample(5))


outputs_input:
 3            <SOS> 여행은 언제나 좋죠. <EOS>
6     <SOS> 다시 새로 사는 게 마음 편해요. <EOS>
10           <SOS> 자랑하는 자리니까요. <EOS>
16        <SOS> 어서 잊고 새출발 하세요. <EOS>
14        <SOS> 돈은 다시 들어올 거예요. <EOS>
Name: A, dtype: object

outputs_target
:  13         혼자를 즐기세요. <EOS>
16    어서 잊고 새출발 하세요. <EOS>
14    돈은 다시 들어올 거예요. <EOS>
3        여행은 언제나 좋죠. <EOS>
2        여행은 언제나 좋죠. <EOS>
Name: A, dtype: object


In [ ]:
# Data Tokenizing
# 각 단어 종류에 대하여 숫자값을 배당한다
# 같은 언어 사이에서의 번역이므로, 어휘 목록을 구성하는 토크나이저는 하나만 필요하다
# 따라서 input과 output을 결합한다
from keras.preprocessing.text import Tokenizer
inputs_series = pd.Series(inputs)                                    # outputs_input 과 같은 Series로 변환한다
inputs_outputs = pd.concat([inputs_series, outputs_input], axis=0)   # input과 output을 결합한다

tokenizer = Tokenizer(num_words=None, char_level=False, lower=False)     # 고빈도 어휘만 사용하려면 num_words에 값을 줄 수 있다. char_level은 False로 해야 한다
tokenizer.fit_on_texts(inputs_outputs)     	                             # inputs와 outputs이 결합된 내용으로 인덱스를 구축한다
word_index = tokenizer.word_index                                        # 단어와 인덱스의 쌍을 가져온다

print('\n전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index))
print('word_index: ', word_index)
print('vocab_size: ', len(word_index))


전체에서 100개의 고유한 토큰을 찾았습니다.
word_index:  {'SOS': 1, 'EOS': 2, 'SNS': 3, '다시': 4, '거예요': 5, '3박4일': 6, '놀러가고': 7, '싶다': 8, 'SD카드': 9, '가끔': 10, '궁금해': 11, '가스불': 12, '여행은': 13, '언제나': 14, '좋죠': 15, '새로': 16, '사는': 17, '게': 18, '마음': 19, '편해요': 20, '시간을': 21, '정하고': 22, '해보세요': 23, '그': 24, '사람도': 25, '그럴': 26, '빨리': 27, '집에': 28, '돌아가서': 29, '끄고': 30, '나오세요': 31, '12시': 32, '땡': 33, '1지망': 34, '학교': 35, '떨어졌어': 36, '정도': 37, 'PPL': 38, '심하네': 39, '망가졌어': 40, '안돼': 41, '맞팔': 42, '왜': 43, '안하지ㅠㅠ': 44, '시간낭비인': 45, '거': 46, '아는데': 47, '매일': 48, '하는': 49, '중': 50, '시간낭비인데': 51, '자꾸': 52, '보게됨': 53, 'SNS보면': 54, '나만': 55, '빼고': 56, '다': 57, '행복해보여': 58, '뭐하는지': 59, '가끔은': 60, '혼자인게': 61, '좋다': 62, '가난한': 63, '자의': 64, '설움': 65, '가만': 66, '있어도': 67, '땀난다': 68, '가상화폐': 69, '쫄딱': 70, '망함': 71, '켜고': 72, '나갔어': 73, '켜놓고': 74, '나온거': 75, '같아': 76, '하루가': 77, '또': 78, '가네요': 79, '위로해': 80, '드립니다': 81, '눈살이': 82, '찌푸려지죠': 83, '잘': 84, '모르고': 85, '있을': 86, '수도': 87, '있어요': 88, '자랑하는': 89, '자리니까요': 90

In [ ]:
# 토크나이저 저장
import os
import pickle

# DATA_OUT_PATH 경로 생성
if os.path.exists(DATA_OUT_PATH + model_name):
    print("{} -- Folder already exists \n".format(DATA_OUT_PATH))
else:
    os.makedirs(DATA_OUT_PATH + model_name, exist_ok=True)
    print("{} -- Folder create complete \n".format(DATA_OUT_PATH))

with open(DATA_OUT_PATH + model_name +"/transformer.pickle", "wb") as file:
  pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)

/content/gdrive/MyDrive/pytest/data/ -- Folder already exists 



In [ ]:
# Data Sequencing
# 배당된 숫자를 이용하여 각 문장의 문자를 숫자로 치환한다
# source 언어 Sequencing
encoder_input = tokenizer.texts_to_sequences(list(inputs))

print('\nResult of encoder_input sequencing: ')
print(inputs[0], encoder_input[0])
print(inputs[1], encoder_input[1])
print(inputs[2], encoder_input[2])


Result of encoder_input sequencing: 
12시 땡! [32, 33]
1지망 학교 떨어졌어 [34, 35, 36]
3박4일 놀러가고 싶다 [6, 7, 8]


In [ ]:
# target 언어 Sequencing
decoder_input = tokenizer.texts_to_sequences(list(outputs_input))
decoder_target = tokenizer.texts_to_sequences(list(outputs_target))

print('\nResult of decoder_input sequencing: ')
print(outputs_input[0], decoder_input[0])
print(outputs_input[1], decoder_input[1])
print(outputs_input[2], decoder_input[2])

print('\nResult of decoder_target sequencing: ')
print(outputs_target[0], decoder_target[0])
print(outputs_target[1], decoder_target[1])
print(outputs_target[2], decoder_target[2])


Result of decoder_input sequencing: 
<SOS> 하루가 또 가네요. <EOS> [1, 77, 78, 79, 2]
<SOS> 위로해 드립니다. <EOS> [1, 80, 81, 2]
<SOS> 여행은 언제나 좋죠. <EOS> [1, 13, 14, 15, 2]

Result of decoder_target sequencing: 
하루가 또 가네요. <EOS> [77, 78, 79, 2]
위로해 드립니다. <EOS> [80, 81, 2]
여행은 언제나 좋죠. <EOS> [13, 14, 15, 2]


In [ ]:
# 문장의 maxlen 설정하기
# source와 target 문장 모두에서의 최대 길이를 구한다
sentence_max_length = inputs_outputs.apply(lambda x: len(x.split())).max()
print('sentence max length: ', sentence_max_length)

sentence max length:  8


In [ ]:
# Data Padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input_pad = pad_sequences(encoder_input, maxlen=sentence_max_length, padding='post')
decoder_input_pad = pad_sequences(decoder_input, maxlen=sentence_max_length, padding='post')
decoder_target_pad = pad_sequences(decoder_target, maxlen=sentence_max_length, padding='post')

In [ ]:
# 타입 확인
print('\nencoder_input_pad shape: ', encoder_input_pad.shape)
print("inputs: ", inputs[1])
print("encoder_input: ", encoder_input[1])
print("encoder_input_pad: ", encoder_input_pad[1])

print('\ndecoder_input_pad shape: ', decoder_input_pad.shape)
print("outputs_input: ", outputs_input[1])
print("decoder_input: ", decoder_input[1])
print("decoder_input_pad: ", decoder_input_pad[1])

print('\ndecoder_target_pad shape: ', decoder_target_pad.shape)
print("outputs_target: ", outputs_target[1])
print("decoder_target: ", decoder_target[1])
print("decoder_target_pad: ", decoder_target_pad[1])


encoder_input_pad shape:  (19, 8)
inputs:  1지망 학교 떨어졌어
encoder_input:  [34, 35, 36]
encoder_input_pad:  [34 35 36  0  0  0  0  0]

decoder_input_pad shape:  (19, 8)
outputs_input:  <SOS> 위로해 드립니다. <EOS>
decoder_input:  [1, 80, 81, 2]
decoder_input_pad:  [ 1 80 81  2  0  0  0  0]

decoder_target_pad shape:  (19, 8)
outputs_target:  위로해 드립니다. <EOS>
decoder_target:  [80, 81, 2]
decoder_target_pad:  [80 81  2  0  0  0  0  0]


관련 라이브러리 임포트

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import enum
import os
import re
import json

from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
# 랜덤 시드 사용
# 실제에서는 이 부분을 제외한다
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

문장 특수 기호 처리

In [ ]:
PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2

# 변수명 변경
index_inputs = encoder_input_pad
index_outputs = decoder_input_pad
index_targets = decoder_target_pad

# prepro_configs 설정
char2idx_dict = word_index
idx2char_dict = {y: x for x, y in word_index.items()}

# dictionary 추가 및 변경
# dict_ex[new_key] = dict_ex[old_key]
# del dict_ex[old_key]
char2idx_dict['<PAD>'] = 0

char2idx_dict['<SOS>'] = char2idx_dict['SOS']
del char2idx_dict['SOS']

char2idx_dict['<END>'] = char2idx_dict['EOS']
del char2idx_dict['EOS']

idx2char_dict[0] = '<PAD>'
idx2char_dict[1] = '<SOS>'
idx2char_dict[2] = '<END>'

prepro_configs = dict({'char2idx':char2idx_dict, 'idx2char':idx2char_dict, 'vocab_size':len(word_index), 'pad_symbol': '<PAD>', 'std_symbol': '<SOS>', 'end_symbol': '<END>'})
print(prepro_configs)

{'char2idx': {'SNS': 3, '다시': 4, '거예요': 5, '3박4일': 6, '놀러가고': 7, '싶다': 8, 'SD카드': 9, '가끔': 10, '궁금해': 11, '가스불': 12, '여행은': 13, '언제나': 14, '좋죠': 15, '새로': 16, '사는': 17, '게': 18, '마음': 19, '편해요': 20, '시간을': 21, '정하고': 22, '해보세요': 23, '그': 24, '사람도': 25, '그럴': 26, '빨리': 27, '집에': 28, '돌아가서': 29, '끄고': 30, '나오세요': 31, '12시': 32, '땡': 33, '1지망': 34, '학교': 35, '떨어졌어': 36, '정도': 37, 'PPL': 38, '심하네': 39, '망가졌어': 40, '안돼': 41, '맞팔': 42, '왜': 43, '안하지ㅠㅠ': 44, '시간낭비인': 45, '거': 46, '아는데': 47, '매일': 48, '하는': 49, '중': 50, '시간낭비인데': 51, '자꾸': 52, '보게됨': 53, 'SNS보면': 54, '나만': 55, '빼고': 56, '다': 57, '행복해보여': 58, '뭐하는지': 59, '가끔은': 60, '혼자인게': 61, '좋다': 62, '가난한': 63, '자의': 64, '설움': 65, '가만': 66, '있어도': 67, '땀난다': 68, '가상화폐': 69, '쫄딱': 70, '망함': 71, '켜고': 72, '나갔어': 73, '켜놓고': 74, '나온거': 75, '같아': 76, '하루가': 77, '또': 78, '가네요': 79, '위로해': 80, '드립니다': 81, '눈살이': 82, '찌푸려지죠': 83, '잘': 84, '모르고': 85, '있을': 86, '수도': 87, '있어요': 88, '자랑하는': 89, '자리니까요': 90, '혼자를': 91, '즐기세요': 92, '돈은': 93, '들어올': 94, '

모델 하이퍼파라미터 정의

In [ ]:
char2idx = prepro_configs['char2idx']               # {'SNS': 3, '다시': 4, ..., , '<PAD>': 0, '<SOS>': 1, '<END>': 2}}의 딕셔너리
end_index = prepro_configs['end_symbol']            # end_index == '<END>'
vocab_size = prepro_configs['vocab_size']
BATCH_SIZE = 2
MAX_SEQUENCE = 25                                   # 최대 시퀀스 길이
EPOCHS = 30
VALID_SPLIT = 0.1

kargs = {'model_name': model_name,
         'num_layers': 2,                           # 사용할 인코더 레이어의 개수
         'd_model': 512,                            # 임베딩 차원(dimension_model): query, key, value에 대한 임베딩 차원
         'num_heads': 8,                            # 어텐션 헤드 수
         'dff': 2048,                               # dimension of Feed Forward Network. 피드 포워드 네트워크 층의 노드 수
         'input_vocab_size': vocab_size,            # 단어 사전의 수
         'target_vocab_size': vocab_size,           # 단어 사전의 수
         'maximum_position_encoding': MAX_SEQUENCE, # 포지션 인코더의 최대 시퀀스 길이
         'end_token_idx': char2idx[end_index],      # char2idx[end_index] == 2. 종료 표지의 인덱스
         'rate': 0.1                                # Dropout에 사용되는 비율
        }

순방향 마스크

In [ ]:
# seq의 값이 padding 0일 때만 1.0을 출력하고, 그 외에는 0을 출력하는 함수
# 마스킹 대상을 1.0으로 만든다. 이후 -1e9라는 작은 수를 곱하고,
# 후에 softmax() 함수를 거치면서 값이 역전된다
# 입력(batch_size, seq_len) --> return(batch_size, 1, 1, seq_len)로 차원 늘림
# 이후 attention의 (batch_size, heads, en/decoder_len, seq_len)에 합산됨
def create_padding_mask(seq):
    mask = tf.cast(tf.math.equal(seq, 0), tf.float32)                                                   
                                                      
    return mask[:, tf.newaxis, tf.newaxis, :]         

In [ ]:
# 우삼각부분만 1로 마스킹 영역을 표시한다
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask                                      # (seq_len, seq_len)

In [ ]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)      # 인코더 패딩 마스크
    dec_padding_mask = create_padding_mask(inp)      # 디코더 두 번째 어텐션 블록에서 사용되는 패딩 마스크

    #print("tf.shape(tar):", tf.shape(tar))
    #print("tf.shape(tar)[1]:", tf.shape(tar)[1])
    #print("tar:\n", tar)

    # 디코더의 첫 번째 어텐션 블록에서 사용되는 마스크
    # 디코더가 받은 데이터를 패딩 처리 이후 순방향 마스킹을 하여 미래의 단어가 참고되지 않게 한다
    # combined_mask는 look_ahead_mask 라는 이름으로 사용될 것이다
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask


# 마스크 실행
enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(index_inputs, index_outputs)

In [ ]:
enc_padding_mask

In [ ]:
look_ahead_mask

In [ ]:
dec_padding_mask

포지셔널 인코딩

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2*i//2)/np.float32(d_model))
  return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis], 
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.floast32)

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  matmul_qk = tf.matmul(q, k, transpost_b=True)

  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrk(dk)

  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

  output = tf.matmul(attention_weights, v)

  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, **kargs):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = kargs['num_heads']
    self.d_model = kargs['d_model']

    assert self.d_model % self.num_heads == 0

    self.depth = self.d_model // self.num_heads

    # query, key, value 가중치 레이어 생성
    self.wq = tf.keras.layers.Dense(kargs['d_model'])
    self.wk = tf.keras.layers.Dense(kargs['d_model'])
    self.wv = tf.keras.layers.Dense(kargs['d_model'])

    self.dense = tf.keras.layers.Dense(kargs['d_model'])
  

  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))

    return tf.transpose(x, perm=[0, 2, 1, 3])

  
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)
    k = self.wk(k)
    v = self.wv(v)

    q = self.split_heads(q, batch_size)
    k = self.split_heads(k, batch_size)
    v = self.split_heads(v, batch_size)

    scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

    output = self.dense(concat_attention)

    return output, attention_weights


In [ ]:
def feed_forward_network(**kargs):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(kargs['dff'], activation='relu'),
      tf.keras.layers.Dense(kargs['d_model'])
  ])

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, **kargs):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(**kargs)
    self.ffn = feed_forward_network(**kargs)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(kargs['rate'])
    self.dropout2 = tf.keras.layers.Dropout(kargs['rate'])

  def call(self, x, mask):
    attn_output, _ = self.mha(x, x, x, mask)
    attn_output = self.dropout1(attn_output)
    out1 = self.layernorm1(x + attn_output)

    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output)

    return out2

In [ ]:
data = np.array([0.006, 0.002, 0.004, 0.009, 0.005, 0.076, 0.007, 0.008, 0.003])
data = data.reshape(-1, 3)
print("data", data)

layer = tf.keras.layers.LayerNormalization(axis=1)
output = layer(data)
print("output", output)

data [[0.006 0.002 0.004]
 [0.009 0.005 0.076]
 [0.007 0.008 0.003]]
output tf.Tensor(
[[ 0.06316139 -0.0631614   0.        ]
 [-0.46261042 -0.55072665  1.013337  ]
 [ 0.03154924  0.06309849 -0.09464776]], shape=(3, 3), dtype=float32)


In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, **kargs):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(**kargs)
    self.mha2 = MultiHeadAttention(**kargs)
    self.ffn = feed_forward_network(**kargs)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.Dropout(kargs['rate'])
    self.dropout2 = tf.keras.Dropout(kargs['rate'])
    self.dropout3 = tf.keras.Dropout(kargs['rate'])

  
  def call(self, x, enc_output, look_ahead_mask, padding_mask):
    attn1, attn_weights_block1 = self,mha1(x, x, x, look_ahead_mask)
    attn1 = self.dropout1(attn1)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
    attn2 = self.dropout2(attn2)
    out2 = self.layernorm2(attn2 + out1)

    ffn_output = self.ffn(out2)
    ffn_output = self.dropout3(ffn_output)
    out3 = self.layernorm3(ffn_output + out2)

    return out3, attn_weights_block1, attn_weights_block2

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, **kargs):
    super(Encoder, self).__init__()

    self.d_model = kargs['d_model']
    self.num_layers = kargs['num_layers']

    # 워드 임베딩 레이어 생성
    self.embedding = tf.keras.layers.Embedding(input_dim=kargs['input_vocab_size'], output_dim=self.d_model)
    self.pos_encoding = positional_encoding(position=kargs['maximum_position_encoding'], d_model=self.d_model)
    self.enc_layers = [EncoderLayer(**kargs) for _ in range(self.num_layers)]
    self.dropout = tf.keras.layers.Dropout(kargs['rate'])

  def call(self, x, mask)    :
    seq_len = tf.shape(x)[1]

    x = self.embewdding(x)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, mask)
    
    return x

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, **kargs):
    super(Decoder, self).__init__()

    self.d_model = kargs['d_model']
    self.num_layers = kargs['num_layers']

    self.embedding = tf.keras.layers.Embedding(input_dim=kargs['target_vocab_size'], output_dim=self.d_model)
    self.pos_encoding = positional_encoding(position=kargs['maximum_position_encoding'], d_model=self.d_model)
    self.dec_layers = [DecoderLayer(**kargs) for _ in range(self.num_layers)]

    self.dropout = tf.keras.layers.Dropout(kargs['rate'])

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, look_ahead_mask, padding_mask)
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
    return x, attention_weights

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, **kargs):
    super(Transformer, self).__init__(name=kargs['model_name'])
    self.end_token_idx = kargs['end_token_idx']

    self.encoder = Encoder(**kargs)
    self.decoder = Decoder(**kargs)

    self.final_layer = tf.keras.layers.Dense(kargs['target_vocab_size'])

  def call(self, x):
    inp, tar = x

    enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
    enc_output = self.encoder(inp, enc_padding_mask)
    dec_output, _ = self.decoder(tar, enc_output, look_ahead_mask, dec_padding_mask)
    final_output = self.final_layer(dec_output)

    return final_output

  def inference(self, x):
    inp = x
    tar = tf.expand_dims([STD_INDEX], axis=0)
    enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
    enc_output = self.encoder(inp, enc_padding_mask)

    predict_tokens = list()
    for t in range(0, MAX_SEQUENCE):
      dec_output, _ = self.decoder(tar, enc_output, look_ahead_mask, dec_padding_mask)
      final_output = self.final_layer(dec_output)
      outputs = tf.argmax(final_output, axis=-1).numpy()
      print("outputs:", outputs)
      pred_token = outputs[0][-1]
      if pred_token == self.end_token_idx:
        break
      predict_tokens.append(pred_token)
      tar = tf.expand_dims([STD_INDEX] + predict_tokens, axis=0)
      _, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
    
    return predict_tokens